<a href="https://colab.research.google.com/github/Akshat-Math/Stock_Price_Prediction_using_Linear_Regression/blob/main/STPRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import svm
from sklearn.linear_model import LinearRegression

In [3]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/data/AMZN.csv')

In [6]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1997-05-15,2.437500,2.500000,1.927083,1.958333,1.958333,72156000
1,1997-05-16,1.968750,1.979167,1.708333,1.729167,1.729167,14700000
2,1997-05-19,1.760417,1.770833,1.625000,1.708333,1.708333,6106800
3,1997-05-20,1.729167,1.750000,1.635417,1.635417,1.635417,5467200
4,1997-05-21,1.635417,1.645833,1.375000,1.427083,1.427083,18853200


In [7]:
df['HL_PCT'] = (df['High'] - df['Adj Close']) / df['Adj Close']   *100.0

In [8]:
df['PCT_CHANGE'] = (df['Adj Close'] - df['Open']) / df['Open']   *100.0

In [9]:
df = df[['Adj Close' , 'HL_PCT' , 'PCT_CHANGE' , 'Volume']]

In [10]:
df.head()

,Adj Close,HL_PCT,PCT_CHANGE,Volume
0,1.958333,27.659596,-19.658133,72156000
1,1.729167,14.457829,-12.169295,14700000
2,1.708333,3.658537,-2.958617,6106800
3,1.635417,7.006348,-5.421686,5467200
4,1.427083,15.328471,-12.738892,18853200


In [11]:
forecast_col = 'Adj Close'
df.fillna(-99999 , inplace = True)

<ipython-input-11-fd4df82471ca>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(-99999 , inplace = True)


In [12]:
forecast_out = int(math.ceil(0.0001*len(df)))

math_ceil(): Rounds to the highest number for example:

           math_ceil(4.5)

it will round to:

            5

then the code will be multiplied by:

           (0.0001*len(df))  

len(df) is basically the size of the dataset which in this case is 6155

           (print (len(df))

In other words we are keeping track of data for 6155 days, now in our case, we will be forecasting what is happening in the future, but obviously, we won't go for 6155 days timeframe but we will take a small dive into the future, in our case it will be 1 day(0.01% of our total data) beyond our last data in our classifier to see what prices will it be on that period.

In [13]:
df["label"] = df[forecast_col].shift(-forecast_out)
print(forecast_out)

1


<ipython-input-13-62e1231c6d1f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = df[forecast_col].shift(-forecast_out)


This formula means that we are shifting the column of our dataset up so what appears in our vision is the stock price after 69 days.

In [14]:
df.head()

,Adj Close,HL_PCT,PCT_CHANGE,Volume,label
0,1.958333,27.659596,-19.658133,72156000,1.729167
1,1.729167,14.457829,-12.169295,14700000,1.708333
2,1.708333,3.658537,-2.958617,6106800,1.635417
3,1.635417,7.006348,-5.421686,5467200,1.427083
4,1.427083,15.328471,-12.738892,18853200,1.395833


In [15]:
X = np.array(df.drop(["label"],axis=1))
X = preprocessing.scale(X)
X_lately = X[-forecast_out:]   #it is the data of last 57 days that has been trained
X = X[:-forecast_out]

In [16]:
df.dropna(inplace=True)

<ipython-input-16-c64f9f573c18>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [17]:
Y = np.array(df["label"])

In [18]:
print(len(X) , len(Y)) #both X and Y should be equal cause fir train and test bhi karna hai nahi to error dega

6154 6154


In [19]:
X_train , X_test , Y_train, Y_test = model_selection.train_test_split(X,Y,test_size = 0.2)

In [20]:
clf = LinearRegression()    #if we use svm.SVR() we can see ki accuracy decreases as hell , try karke dekhna

READ DOCUMENTATION FOR UNDERSTANDING PARAMETERS THAT CAN BE PASSED UNDER DIFFERENT SITUATIONS.

In [21]:
clf.fit(X_train , Y_train)

LinearRegression()

In [26]:
accuracy = clf.score(X_test , Y_test)
print(accuracy)

0.9994867781251706


We trained and tested on two different datasets cause if we trained and tested on same data then the machine would already know the answer to the trainig data and then give out 100% accuracy similar to if we knew the questions in practise and exams are same then there would we 100% full marks.

Here the accuracy is 99.94%.

In [23]:
print(forecast_out)

1


so here we have the forecast for next 1 day with 98.02% accuracy.


In [24]:
forecast_set = clf.predict(X_lately)

In [25]:
print(forecast_set,accuracy,forecast_out)

[3398.87171377] 0.9994867781251706 1
